In [16]:
!pip install roboflow

In [17]:
!pip install easyocr

In [18]:
!pip install pytesseract

In [19]:
!pip install scikit-learn

In [20]:
import easyocr
import cv2
import pytesseract
import tensorflow as tf
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from PIL import Image

In [21]:
from roboflow import Roboflow
rf = Roboflow(api_key="f77FoHccQ1RtNOK1tFRo")
project = rf.workspace("jakob-awn1e").project("receipt-or-invoice")
version = project.version(2)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Receipt-or-Invoice-2 in folder:: 100%|██████████| 4325/4325 [00:01<00:00, 4226.14it/s]


In [12]:
# Preprocessing
def preprocess_images(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    return image

In [13]:
def label_data(data_path):
    images = []
    labels = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):  
                file_path = os.path.join(root, file)
                label = os.path.basename(os.path.dirname(file_path)) 
                image = preprocess_images(file_path)
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

In [14]:
# Load dataset
train_path = "/content/Receipt-or-Invoice-2/train"
val_path = "/content/Receipt-or-Invoice-2/valid"
test_path = "/content/Receipt-or-Invoice-2/test"

In [15]:
X_train, y_train = label_data(train_path)
X_val, y_val = label_data(val_path)
X_test, y_test = label_data(test_path)

In [ ]:
import numpy as np
# Combining labels from all three datasets
all_labels = np.concatenate((y_train, y_val, y_test))

# Encode labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Re-encode labels for each dataset
y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)